<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/07_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門 (3)
* チュートリアルをほぼそのまま使う。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* テキスト分類をPyTorchで実装する。
* torchtextの使い方も併せて学ぶ。
 * https://pytorch.org/text/stable/index.html

## 準備

* ランタイムの設定でGPUを選択しておこう。

* torchdataを使うために必要なパッケージのインストール

In [ ]:
!pip install 'portalocker>=2.0.0'

**ここでランタイムを再起動すること。**

## PyTorchのインポート

In [ ]:
import torch

# 再現性の確保
torch.manual_seed(0)

# 使用するデバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## `torchtext.datasets`

* torchtextに準備された仕組みを利用して、データセットを準備する。
* 今回はAG_NEWSというテキスト分類用のデータセットを使う。

In [ ]:
from torchtext.datasets import AG_NEWS

train_ = AG_NEWS(split="train")

* データセットは以下のように特殊な型を持つが・・・

In [ ]:
type(train_)

torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe

* iter関数でiteratorに変換できる。
 * ちょっとデータセットの中身を見てみたい時は、こうすると便利。

In [ ]:
train_iter = iter(train_)
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [ ]:
next(train_iter)

(3,
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')

In [ ]:
next(train_iter)

(3,
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.")

## 前処理

### 語彙集合の作成
 * 語彙を確定させるときは、訓練データだけを使うこと。
 * `build_vocab_from_iterator`についてはドキュメントを参照。
  * https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# トークナイザを準備する
tokenizer = get_tokenizer("basic_english")

# トークン化をおこなう関数
def yield_tokens(data_):
  for _, text in data_:
    yield tokenizer(text)

# 訓練データをもとに語彙集合を作成（ほんの少し時間がかかる）
vocab = build_vocab_from_iterator(yield_tokens(train_), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
type(vocab)

torchtext.vocab.vocab.Vocab

* 単語トークンの列が整数の列に変換されることを確認する。

In [ ]:
vocab(['here', 'is', 'an', 'example'])

[475, 21, 30, 5297]

In [ ]:
len(vocab)

95811

### 前処理のパイプラインの定義
 * テキストは、トークン化し、そして、idの列に変換する。
 * クラスラベルは、整数化し、1を引くことで値が0から始まるようにする。

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [ ]:
text_pipeline('Here is an example.')

[475, 21, 30, 5297, 1]

In [ ]:
label_pipeline('10')

9

### Collate関数
* サンプルに前処理を施してミニバッチを作ることを、collateする、と言う。
* collate関数の中で、先ほど定義した前処理パイプラインを呼び出している。
* 今回は、同じミニバッチに含まれるテキストをすべてつなげてしまう。
* `offsets`は各テキストが何トークン目から始まるかを表す。
* このcollate関数は、後でDataLoaderを作るときに使う。

In [ ]:
def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]
  for _label, _text in batch:
    label_list.append(label_pipeline(_label))
    processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  text_list = torch.cat(text_list)
  return label_list.to(device), text_list.to(device), offsets.to(device)

## DataLoader

* 訓練データとテストデータを用意する。

In [ ]:
train_ = AG_NEWS(split="train")
test_ = AG_NEWS(split="test")

* 上のセルでやったことは、実は1行で以下のように書ける。

In [ ]:
train_, test_ = AG_NEWS()

* Dataset型に変換する。
 * torchtextに用意されている`to_map_style_dataset`関数を使う。

In [ ]:
from torchtext.data.functional import to_map_style_dataset

train_dataset = to_map_style_dataset(train_)
test_dataset = to_map_style_dataset(test_)

* 訓練データを２分割して検証データを作成する。

In [ ]:
from torch.utils.data.dataset import random_split

# 訓練データから5%を取って検証データとする。
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

### DataLoader
* 訓練データ、検証データ、テストデータのDataLoaderを作る。
* collate関数の使い方に注目。

In [ ]:
from torch.utils.data import DataLoader

# ミニバッチのサイズを適当に決める
BATCH_SIZE = 64

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

## モデル

### `nn.EmbeddingBag`
* 全トークンのembeddingの平均（または和）を一挙に求めるlayer。

In [ ]:
from torch import nn

embedding = nn.EmbeddingBag(len(vocab), 8, sparse=False)

* 本当に平均を求めているかを確認する。

In [ ]:
text = "language models"
input = torch.tensor(text_pipeline(text), dtype=torch.int64)
offsets = torch.tensor([0], dtype=torch.int64)
embedding(input=input, offsets=offsets)

tensor([[ 0.3720,  0.9064, -1.4657, -0.1584,  0.1959,  1.2021, -0.5094, -0.0792]],
       grad_fn=<EmbeddingBagBackward0>)

In [ ]:
text = "language"
input = torch.tensor(text_pipeline(text), dtype=torch.int64)
offsets = torch.tensor([0], dtype=torch.int64)
output1 = embedding(input=input, offsets=offsets)
output1

tensor([[ 0.9283,  0.8008, -1.3259,  1.5126,  0.5105,  1.4376, -0.3588, -0.2826]],
       grad_fn=<EmbeddingBagBackward0>)

In [ ]:
text = "models"
input = torch.tensor(text_pipeline(text), dtype=torch.int64)
offsets = torch.tensor([0], dtype=torch.int64)
output2 = embedding(input=input, offsets=offsets)
output2

tensor([[-0.1843,  1.0119, -1.6056, -1.8294, -0.1188,  0.9665, -0.6600,  0.1241]],
       grad_fn=<EmbeddingBagBackward0>)

In [ ]:
(output1 + output2) / 2

tensor([[ 0.3720,  0.9064, -1.4657, -0.1584,  0.1959,  1.2021, -0.5094, -0.0792]],
       grad_fn=<DivBackward0>)

* offsetはテキストの切れ目を表す。
* 複数のテキストをつなげたままベクトル化できる。
 * メモリの効率も時間的な効率も良い。

In [ ]:
text = "language models text classification"
input = torch.tensor(text_pipeline(text), dtype=torch.int64)
offsets = torch.tensor([0, 2], dtype=torch.int64)
embedding(input=input, offsets=offsets)

tensor([[ 0.3720,  0.9064, -1.4657, -0.1584,  0.1959,  1.2021, -0.5094, -0.0792],
        [ 0.2176, -0.3015,  0.2869,  0.4253, -0.2544,  0.1135, -0.3574,  1.1874]],
       grad_fn=<EmbeddingBagBackward0>)

### 分類モデル
* `nn.Module`を継承して自前のクラスを定義する。


In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, num_class):
    super(TextClassificationModel, self).__init__()
    # 埋め込み層
    self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
    # 分類用の全結合層
    self.fc = nn.Linear(embed_dim, num_class)
    # 自前の重み初期化関数を呼び出す
    self.init_weights()

  # 自前の重み初期化関数
  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  # forward pass
  def forward(self, text, offsets):
    embedded = self.embedding(text, offsets)
    return self.fc(embedded)

* 訓練データを使ってクラスの個数を調べる。

In [ ]:
train_iter = AG_NEWS(split="train")
num_class = len(set([label for (label, text) in train_iter]))

* 重要な定数を変数にセットする。

In [ ]:
# 語彙サイズ
vocab_size = len(vocab)

# 埋め込みベクトルの次元
emsize = 64

* モデルのインスタンスを作成しGPUへ送る。
 * 上で値をセットした変数を使って初期化している。

In [ ]:
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

## 訓練に使うヘルパ関数

In [ ]:
import time

def train(dataloader):
  model.train()
  total_acc, total_count = 0, 0
  log_interval = 500
  start_time = time.time()

  for idx, (label, text, offsets) in enumerate(dataloader):
    optimizer.zero_grad()
    predicted_label = model(text, offsets)
    loss = criterion(predicted_label, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    optimizer.step()
    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)
    if idx % log_interval == 0 and idx > 0:
      elapsed = time.time() - start_time
      print(
          "| epoch {:3d} | {:5d}/{:5d} batches "
          "| accuracy {:8.3f}".format(
              epoch, idx, len(dataloader), total_acc / total_count
          )
      )
      total_acc, total_count = 0, 0
      start_time = time.time()

## 評価に使うヘルパ関数

In [ ]:
def evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
      predicted_label = model(text, offsets)
      loss = criterion(predicted_label, label)
      total_acc += (predicted_label.argmax(1) == label).sum().item()
      total_count += label.size(0)
  return total_acc / total_count

## 学習のハイパーパラメータ

In [ ]:
EPOCHS = 10  # epoch
LR = 5  # learning rate

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

total_accu = None

for epoch in range(1, EPOCHS + 1):
  epoch_start_time = time.time()
  train(train_dataloader)
  accu_val = evaluate(valid_dataloader)
  if total_accu is not None and total_accu > accu_val:
    scheduler.step()
  else:
    total_accu = accu_val
  print("-" * 59)
  print(
      "| end of epoch {:3d} | time: {:5.2f}s | "
      "valid accuracy {:8.3f} ".format(
          epoch, time.time() - epoch_start_time, accu_val
      )
  )
  print("-" * 59)

In [ ]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

In [ ]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}


def predict(text, text_pipeline):
  with torch.no_grad():
    text = torch.tensor(text_pipeline(text))
    output = model(text, torch.tensor([0]))
    return output.argmax(1).item() + 1


ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" % ag_news_label[predict(ex_text_str, text_pipeline)])

# 本日の課題
* モデルやoptimizerやschedulerを変更して、validation setを使ってチューニングしよう。
* 最後に、自分で選択した設定を使って、test set上で評価しよう。